## Import EDA and Beyond Library from path

In [1]:
# import importlib.util
# import sys

# spec = importlib.util.spec_from_file_location('eda_mod', '/Users/fredaxin/oss/EDA_and_beyond/eda_and_beyond.py')
# module = importlib.util.module_from_spec(spec)
# sys.modules['eda_mod'] = module
# spec.loader.exec_module(module)

## Import other Libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

## Read in data

In [3]:
top_25 = pd.read_csv('./processed_data_1/cleaned_building_permits&pop_top_25.csv')
york = pd.read_csv('./processed_data_1/cleaned_building_permits&pop_york_region.csv')

In [4]:
top_25.head()

,csd,province_or_territory,year,value_nonresidential_commercial,value_nonresidential_industrial,value_nonresidential_institutional,value_residential_single_single,value_residential_single_mobile,value_residential_single_cottage,value_residential_multiple_double,value_residential_multiple_row,value_residential_multiple_apartment,units_residential_single_single,units_residential_single_mobile,units_residential_single_cottage,units_residential_multiple_double,units_residential_multiple_row,units_residential_multiple_apartment,population
0,Brampton,Ontario,2000,131629.0,90918.0,18058.0,484842.0,0.0,0.0,162743.0,83027.0,17132.0,2907,0,0,1431,871,177,325428
1,Brampton,Ontario,2001,72867.0,44095.0,82349.0,563840.0,0.0,0.0,132851.0,49349.0,8000.0,3269,0,0,1109,475,98,325428
2,Brampton,Ontario,2002,66005.0,28119.0,139133.0,894218.0,0.0,0.0,120012.0,44770.0,0.0,5013,0,0,945,409,0,325428
3,Brampton,Ontario,2003,95332.0,84011.0,68715.0,1101153.0,0.0,0.0,209710.0,53412.0,1740.0,4825,0,0,1374,457,15,325428
4,Brampton,Ontario,2004,174868.0,77287.0,254750.0,1723129.0,0.0,0.0,196322.0,102800.0,17000.0,7359,0,0,1287,823,98,325428


In [5]:
york.head()

,csd,province_or_territory,year,value_nonresidential_commercial,value_nonresidential_industrial,value_nonresidential_institutional,value_residential_single_single,value_residential_single_mobile,value_residential_single_cottage,value_residential_multiple_double,value_residential_multiple_row,value_residential_multiple_apartment,units_residential_single_single,units_residential_single_mobile,units_residential_single_cottage,units_residential_multiple_double,units_residential_multiple_row,units_residential_multiple_apartment,population
0,Aurora,Ontario,2000,3000.0,2220.0,6994.0,10785.0,0.0,0.0,3884.0,7491.0,0.0,69,0,0,48,90,0,40167
1,Aurora,Ontario,2001,5000.0,1811.0,210.0,15251.0,0.0,0.0,5770.0,10240.0,0.0,127,0,0,68,113,0,40167
2,Aurora,Ontario,2002,1725.0,0.0,5615.0,48410.0,0.0,0.0,364.0,8905.0,0.0,383,0,0,4,85,0,40167
3,Aurora,Ontario,2003,5175.0,3931.0,6015.0,111687.0,0.0,0.0,21268.0,32658.0,0.0,710,0,0,163,287,0,40167
4,Aurora,Ontario,2004,101201.0,318.0,5500.0,41532.0,0.0,0.0,552.0,25998.0,0.0,171,0,0,4,175,0,40167


## Check General Information and Data Types

In [6]:
top_25.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 19 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   csd                                   500 non-null    object 
 1   province_or_territory                 500 non-null    object 
 2   year                                  500 non-null    int64  
 3   value_nonresidential_commercial       500 non-null    float64
 4   value_nonresidential_industrial       500 non-null    float64
 5   value_nonresidential_institutional    500 non-null    float64
 6   value_residential_single_single       500 non-null    float64
 7   value_residential_single_mobile       500 non-null    float64
 8   value_residential_single_cottage      500 non-null    float64
 9   value_residential_multiple_double     500 non-null    float64
 10  value_residential_multiple_row        500 non-null    float64
 11  value_residential_m

In [7]:
york.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 19 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   csd                                   140 non-null    object 
 1   province_or_territory                 140 non-null    object 
 2   year                                  140 non-null    int64  
 3   value_nonresidential_commercial       140 non-null    float64
 4   value_nonresidential_industrial       140 non-null    float64
 5   value_nonresidential_institutional    140 non-null    float64
 6   value_residential_single_single       140 non-null    float64
 7   value_residential_single_mobile       140 non-null    float64
 8   value_residential_single_cottage      140 non-null    float64
 9   value_residential_multiple_double     140 non-null    float64
 10  value_residential_multiple_row        140 non-null    float64
 11  value_residential_m

In [8]:
top_25.shape

(500, 19)

In [9]:
york.shape

(140, 19)

In [10]:
top_25.describe().T

,count,mean,std,min,25%,50%,75%,max
year,500.0,2009.500000,5.772056,2000.0,2004.75,2009.5,2014.25,2019.0
value_nonresidential_commercial,500.0,164335.322946,205884.415825,6221.0,50986.00,98557.5,191020.75,2004101.0
value_nonresidential_industrial,500.0,33715.749066,51577.550644,0.0,5214.25,15145.5,40691.75,510295.0
value_nonresidential_institutional,500.0,77533.541940,155884.929863,0.0,13269.00,35507.0,82184.00,1891026.0
value_residential_single_single,500.0,309417.767960,327159.268273,5650.0,95977.25,202949.0,390818.50,1833578.0
value_residential_single_mobile,500.0,275.646000,1152.514757,0.0,0.00,0.0,0.00,10540.0
value_residential_single_cottage,500.0,6.926000,44.898689,0.0,0.00,0.0,0.00,612.0
value_residential_multiple_double,500.0,35818.388000,54566.615879,0.0,3280.75,14185.0,40732.75,337364.0
value_residential_multiple_row,500.0,74591.290000,89020.968540,0.0,14690.00,41756.0,94143.25,593809.0
value_residential_multiple_apartment,500.0,290480.329606,472696.598677,0.0,48170.00,136653.0,309551.00,3802706.0


In [11]:
york.describe().T

,count,mean,std,min,25%,50%,75%,max
year,140.0,2009.500000,5.786986,2000.0,2004.75,2009.5,2014.25,2019.0
value_nonresidential_commercial,140.0,12587.728571,20651.387058,0.0,1912.50,5432.0,15459.50,181350.0
value_nonresidential_industrial,140.0,3674.542857,5619.503320,0.0,343.75,1521.5,4366.50,29357.0
value_nonresidential_institutional,140.0,8464.892857,14926.244551,0.0,19.50,1175.0,10319.50,85104.0
value_residential_single_single,140.0,86704.950007,97394.612113,9196.0,26914.50,48398.0,120190.25,678064.0
value_residential_single_mobile,140.0,10.928571,64.606127,0.0,0.00,0.0,0.00,500.0
value_residential_single_cottage,140.0,2.500000,23.262190,0.0,0.00,0.0,0.00,270.0
value_residential_multiple_double,140.0,4558.250000,8063.905434,0.0,0.00,1180.0,4154.00,44674.0
value_residential_multiple_row,140.0,14881.985714,24626.773653,0.0,0.00,5107.5,17311.00,123909.0
value_residential_multiple_apartment,140.0,9556.957143,21611.458344,0.0,0.00,0.0,7390.00,132969.0


## Feature Engineering

### Generate totol unit and value

In [12]:
def generate_sum(df):
    val_cols = [i for i in list(df.columns) if i.startswith('value')]
    unit_cols = [i for i in list(df.columns) if i.startswith('units')]
    val_sum_res_cols = [i for i in list(df.columns) if i.startswith('value_residential')]
    val_sum_nonres_cols = [i for i in list(df.columns) if i.startswith('value_nonresidential')]
    
    df['val_sum'] = df[val_cols].sum(axis=1)
    
    df['val_sum_res'] = df[val_sum_res_cols].sum(axis=1)
    df['val_sum_nonres'] = df[val_sum_nonres_cols].sum(axis=1)
    
    # For units, the data source only provides number of units for residential
    df['unit_sum_res'] = df[unit_cols].sum(axis=1)
    return df

In [13]:
generate_sum(top_25)
generate_sum(york);

### Generate unit and value per capita

In [14]:
def generate_pc(df):
    cols = [i for i in list(df.columns) if i not in ['csd', 'province_or_territory', 'year', 'population']]
    for col in cols:
        df[f'pc_{col}'] = df[col] / df['population']
    return df

In [15]:
generate_pc(top_25)
generate_pc(york);

### Generate percentage of each unit and value types

In [16]:
def generate_percent_unit(df):
    cols = [i for i in list(df.columns) if i.startswith('units')]
    for col in cols:
        df[f'percent_{col}'] = df[col] / df['unit_sum_res']
    return df

In [17]:
generate_percent_unit(top_25)
generate_percent_unit(york);

In [18]:
# # Check if percentages add up to 100%
# percent_units_cols = [i for i in list(top_25.columns) if i.startswith('percent_units')]
# top_25['sum'] = top_25[percent_units_cols].sum(axis=1)
# pd.set_option('display.max_rows', 500)
# top_25[top_25['sum'] != 1.0]

In [19]:
def generate_percent_value(df):
    cols = [i for i in list(df.columns) if i.startswith('value')]
    for col in cols:
        df[f'percent_{col}'] = df[col] / df['val_sum']
    return df

In [20]:
generate_percent_value(top_25)
generate_percent_value(york);

In [21]:
# # Check if percentages add up to 100%
# percent_val_cols = [i for i in list(top_25.columns) if i.startswith('percent_value')]
# top_25['sum'] = top_25[percent_val_cols].sum(axis=1)
# pd.set_option('display.max_rows', 500)
# top_25[top_25['sum'] != 1.0]

### Generate rolling sum and avg

#### Top 25

In [22]:
# Reference: https://stackoverflow.com/questions/53339021/python-pandas-calculate-moving-average-within-group
top_25['rolling_avg_val'] = top_25.groupby('csd').rolling(3)['val_sum'].mean().reset_index(drop=True)

In [23]:
top_25['rolling_sum_val'] = top_25.groupby('csd').rolling(3)['val_sum'].sum().reset_index(drop=True)

In [24]:
top_25['rolling_avg_unit'] = top_25.groupby('csd').rolling(3)['unit_sum_res'].mean().reset_index(drop=True)

In [25]:
top_25['rolling_sum_unit'] = top_25.groupby('csd').rolling(3)['unit_sum_res'].sum().reset_index(drop=True)

In [26]:
# pd.set_option('display.max_rows', 500)
# top_25.head(25).T

#### York region

In [27]:
# Reference: https://stackoverflow.com/questions/53339021/python-pandas-calculate-moving-average-within-group
york['rolling_avg_val'] = york.groupby('csd').rolling(3)['val_sum'].mean().reset_index(drop=True)

In [28]:
york['rolling_sum_val'] = york.groupby('csd').rolling(3)['val_sum'].sum().reset_index(drop=True)

In [29]:
york['rolling_avg_unit'] = york.groupby('csd').rolling(3)['unit_sum_res'].mean().reset_index(drop=True)

In [30]:
york['rolling_sum_unit'] = york.groupby('csd').rolling(3)['unit_sum_res'].sum().reset_index(drop=True)

In [31]:
# york.head(25)

## Export as csv

In [32]:
# top_25.to_csv("eda_top_25.csv", index=False)
# york.to_csv("eda_york.csv", index=False)